In [ ]:
import cv2
import numpy as np
import pyautogui
from tensorflow import keras 
from ultralytics import YOLO

In [ ]:
yoloPath = "Yolos/Insects/BIG dataset/train34/weights/best.pt"
beePath = "../Classifiers/classifiers/beeOrNot.keras"
waspPath = "../Classifiers/classifiers/waspOrNot.keras"
yoloModel = YOLO(yoloPath)
yoloModel.to(device="cuda")
beeModel = keras.models.load_model(beePath)
waspModel = keras.models.load_model(waspPath)

In [ ]:
threshold = 0.3
inputSize = (224,224)

In [ ]:
screenWidth, screenHeight = pyautogui.size()

region = (0, 0, screenWidth, screenHeight)

windowName = "YOLO Detection with Custom Classification"
cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)
cv2.resizeWindow(windowName, 800, 600)

while True:
    screenshot = pyautogui.screenshot(region=region)

    frame = np.array(screenshot)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    results = yoloModel.track(source=frame, conf=threshold, verbose=False)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  
        confidences = result.boxes.conf.cpu().numpy()  


        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)

            x1 = max(0, min(x1, frame.shape[1] - 1))
            y1 = max(0, min(y1, frame.shape[0] - 1))
            x2 = max(0, min(x2, frame.shape[1] - 1))
            y2 = max(0, min(y2, frame.shape[0] - 1))

            if x2 > x1 and y2 > y1:
                
                croppedRegion = frame[y1:y2, x1:x2]
                
                croppedInput = cv2.resize(croppedRegion, inputSize)
                croppedInput = cv2.cvtColor(croppedInput, cv2.COLOR_RGB2BGR)
                croppedInput = np.expand_dims(croppedInput, axis=0)  

                predictions = beeModel.predict(croppedInput, verbose=False)
                predictedClass = "Bee"
                confidenceScore = np.max(predictions)
                if confidenceScore < 0.7:
                    predictions = waspModel.predict(croppedInput, verbose=False)
                    predictedClass = "Wasp"
                    confidenceScore = np.max(predictions)
                    if confidenceScore < 0.7:
                        predictedClass = "Other"
                        confidenceScore = 1.0 - confidenceScore
                    
                    
                label = f"Class {predictedClass} ({confidenceScore:.2f}), Yolo {confidences[i]:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  

    cv2.imshow(windowName, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()